In [17]:
import os
import random

NameError: name 'player' is not defined

In [2]:
class Player:
    
    def __init__(self, name):
        self.name = name
        self.board = Board()
        self.hits_list = []
        self.create_ships()
        print_separator()
        print(f"Player {self.name} created! \nNow positionning the ships...")
        self.place_ships()
 
    def create_ships(self):
        self.carrier = Ship("Carrier", 5)
        self.battleship = Ship("Battleship", 4)
        self.cruiser = Ship("Cruiser", 3)
        self.submarine = Ship("Submarine", 3)
        self.destroyer = Ship("Destroyer", 3)
        self.ships = [self.carrier, self.battleship, self.cruiser, self.submarine, self.destroyer]
      
    def place_ships(self):
        for ship in self.ships:
            if self.place_ship(ship):
                self.add_ship(ship, ship.coords)
            display_board(self, self.board.displayed_input)
        return True
   
    def place_ship(self, ship):
        print_separator()
        print(f"Placing {self.name}'s {ship.model} of size {ship.size}!")
        if not debug_mode:
            user_input = input("Please enter coords (A-Z) (0-9) (right or bottom): ")
        ###improvement####
        else:
            user_input = debug_pattern_place_ships[self.ships.index(ship)] # DEBUG
        coords_input = user_input.split(" ")
        i = coords_input[0]
        j = int(coords_input[1])
        d = coords_input[2]
        ship.update_coords((i,j), d)
        if not self.check_place_ship(ship):
                print("Please retry")
                self.place_ship(ship)
        else:
            self.board.update_filled(ship.coords)
        return True
    
    def add_ship(self, ship, coords):
        for coord in coords:
            self.board.displayed_input[coord[0]][coord[1]] = "X"
    
    def check_place_ship(self, ship):
        if ship.origin[0] not in ('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'):
        ###improvement###
            print("Wrong input: first digit (line) must be within A-J.")
            return False
        if ship.origin[1] > 9:
            print("Wrong input: first digit (column) must be less than 10.")
            return False
        for coord in ship.coords:
            if tuple(coord) not in base_board:
                print("The ship does not fit on the board.")
                return False
            if tuple(coord) in self.board.filled:
                print("Overlap.")
                return False
        return True

[['-', '-', '-', '-', '-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-']]

In [18]:
class Ship:
    
    def __init__(self, model, size):
        self.model = model
        self.size = size
        self.life = size
        self.origin = []
        self.coords=[]
    
    def update_coords(self, origin, direction):
        self.coords=[]
        if direction == "bottom" or direction == "b":
            for i in range(self.size):
                self.coords.append((chr(ord('@')+(list(origin)[0]+i+1)), list(origin)[1])) # Going bottom: increment line, same column
        ###improvement###  
        # chr(ord('@')+number) method found but doesn't seem to work, I must made a mistake and missed to change the other parts of the code
        else:
            for i in range(self.size):
                self.coords.append((chr(ord('@')+(list(origin)[0]+1)), list(origin)[1]+i)) # Going right: same line, increment column
        ###improvement###  DOES NOT WORK
        self.update_origin(self.coords)
    
    def update_origin(self, coords):
        self.origin = chr(ord('@')+coords[0])
        ###improvement###  DOES NOT WORK

In [19]:
class Board:
    
    def __init__(self):
        self.displayed_input = [['-' for i in range(10)] for j in range(10)]
        self.displayed_to_opponent = [['-' for i in range(10)] for j in range(10)]
        self.filled = []
    
    def update_displayed_to_opponent(self, coords, hit):
        if hit:
            self.displayed_to_opponent[coords[0]][coords[1]] = "X"
        else:
            self.displayed_to_opponent[coords[0]][coords[1]] = "O"
                    
    def update_filled(self, coords):
        self.filled.extend([tuple(coord) for coord in coords])

In [20]:
def print_separator():
    print("\n##########################################\n")

def display_board(player, board_to_display):
    print("\n" + f"Board of {player.name}:" + "\n")
    print("\n".join([" ".join(board_to_display[i]) for i in range(10)]))
    
def start_strikes():
    display_board(defender, defender.board.displayed_to_opponent)
    while True:
        strike_result = strike()
        if strike_result == False:
            break
    if flag_game_active:
        switch()
       
def strike():
    print_separator()
    while True:
        if not debug_mode:
            attacker_input = input("Please enter coords (A-Z) (0-9) to strike: ")
            ###improvement###
        else: # DEBUG
            global debug_pattern_strikes
            attacker_input = debug_pattern_strikes[0]
            debug_pattern_strikes.remove(debug_pattern_strikes[0])
        coords_attacker_input = check_strike_input(attacker_input)
        if coords_attacker_input != False:
            break
    attacker.hits_list.append(coords_attacker_input)
    return inflict_damages(coords_attacker_input) # Return FALSE if not damages inflicted

def check_strike_input(attacker_input):
    if len(attacker_input) == 3: # Input format should be '0 0' hence length must be 3
        try:
            coords_attacker_input = tuple(int(i) for i in attacker_input.split(" "))
        except:
            print("Wrong entry: should be (0-9) (0-9), e.g. 0 0")
            return False
    else:
        print("Wrong entry: should be (0-9) (0-9), e.g. 0 0")
        return False
    if coords_attacker_input in attacker.hits_list:
        print("You already hit this spot, please try again!")
        return False
    else:
        return coords_attacker_input
           
def inflict_damages(coords_attacker_input):
    if coords_attacker_input in defender.board.filled:
        defender.board.update_displayed_to_opponent(coords_attacker_input, True)
        for ship in defender.ships:
            if coords_attacker_input in ship.coords:
                ship.life -= 1
                if ship.life == 0:
                        print(f"{ship.model} sunk!")
                        display_board(defender, defender.board.displayed_to_opponent)
                        defender.ships.remove(ship)
                        if defender.ships == []:
                            endgame()
                            return False
                else:
                    print(f"{ship.model} hit! Still {ship.life}/{ship.size} to go to sink it!")
                    display_board(defender, defender.board.displayed_to_opponent)
        return True
    else:
        print("Miss!")
        defender.board.update_displayed_to_opponent(coords_attacker_input, False)
        display_board(defender, defender.board.displayed_to_opponent)
        return False

def switch():
    global attacker, defender
    attacker, defender = defender, attacker
    print_separator()
    print(f"\nNow {attacker.name} attacks!")
    start_strikes()

def endgame():
    global flag_game_active
    print(f"\nYou sunk the last ship of {defender.name}! {attacker.name} wins!")
    flag_game_active = False

def start_game():
    select_first_attacker()
    start_strikes()
    
def select_first_attacker():
    global attacker, defender
    attacker = random.choice([p1, p2])
    if attacker == p1:
        defender = p2
    else:
        defender = p1
    print_separator()
    print(f"\nThe first attacker is {attacker.name}!")

In [21]:
# SUGGESTED IMPROVEMENTS
#
# Change coordinates input system from '0 0 / 9 9' to 'A1 / J10'
# Show A...J and 1...10 when board is displayed
# Improve input checks into a single function input_check(prompt_message, rule regex)
# Comment, reorder & clean code

base_board = [(i,j) for i in range(10) for j in range(10)]
flag_game_active = True

# DEBUG mode (computer plays automatically)
debug_mode = False # Set to True for computer to play all game automatically
debug_pattern_place_ships = ["0 0 r", "1 1 r", "2 2 r", "3 3 r", "4 4 r"]
debug_pattern_strikes = ["0 0", "0 1", "5 5","0 0", "0 1", "0 2", "0 3", "0 4", "1 1", "1 2", "1 3", "1 4", "2 2", "5 5", "0 2", "0 3", "0 4", "1 1", "1 2", "1 3", "1 4", "2 2", "2 3", "2 4", "3 3", "3 4", "3 5", "4 4", "4 5", "4 6"]

print("----------------")
print("BATTLESHIPS GAME")
print("----------------")

if not debug_mode:
    p1 = Player(input("\nPlease enter your name: "))
    p2 = Player(input("\nPlease enter your name: "))
else: # DEBUG
    p1 = Player("Ting")
    p2 = Player("Eric")

start_game()

----------------
BATTLESHIPS GAME
----------------


NameError: name 'Player' is not defined